日本語wikiからコーパスを作成するスクリプトです.<br>
https://dumps.wikimedia.org/jawiki/latest/ <br>
こちらのサイトから最新版の"pages-articles"のアドレスを手に入れてください. <br>

ダンプデータには不要なマークアップなどが含まれているので、取り除くためのテキストクリーニング用のスクリプトをgitから持ってきます

日本語wikiに対してテキストクリーニングを実行します

テキストに前処理を加えた上で,複数のtxtファイルをひとつに結合します

In [6]:
import glob
from bs4 import BeautifulSoup

with open('./data/chatdata/chatdata-sample.txt','w') as f:
#with open('./tmp.txt','w') as f:
    for directory in glob.glob('./extracted/*'):
        for name in glob.glob(directory+'/*'):
            with open(name, 'r') as r:
                for line in r:
                    # titleを削除する
                    if '<doc ' in line:
                        next(r)
                        next(r)
                    elif '</doc>' in line:
                        f.write('\n')
                        continue
                    else:
                        # 空白・改行削除、大文字を小文字に変換
                        text = BeautifulSoup(line.strip()).text.lower()
                        f.write(text)

/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.imdb.com/title/tt0115369/episodes" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://slither.io/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://www.youtube.com/watch?v=wnwQcr8tnAw" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind th

/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.bullerdc.govt.nz/main/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://yama-machi.beblog.jp/sakumab/2010/02/post-46ce.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/schunske/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://gekinobi.futureartist.net" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the

KeyboardInterrupt: 

ここからはBERTのトレーニング用にテキストファイルを整形していきます.<br>
文章を単語ごとに分割し, ひとつの単元の中に偶数個の文章が含まれるように調整します.

In [7]:
import linecache
import random
import MeCab

In [20]:
random.seed(42)
#filename = 'tmp.txt'
filename = './data/chatdata/chatdata-sample.txt'
#save_file = 'even_rows100M.txt'
save_file = './data/chatdata/chatdata-sample-after.txt'
LIMIT_BYTE = 100000000 # 100Mbyte
# t = MeCab.Tagger('-Owakati') # Neologdを辞書に使っている人場合はそちらを使用するのがベターです
#t = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/ -Owakati')
t = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/ -Owakati')

def get_byte_num(s):
    return len(s.encode('utf-8'))

In [31]:
#with open(save_file, 'w') as f:
with open(save_file, 'w') as f:
    count_byte = 0
    with open(filename) as r:
        for text in r:
            print('{} bytes'.format(count_byte))
            #text = t.parse(text).strip()
            # 一文ごとにタブで分割すればいいのではないか
            #text = text.split('。')
            text = text.split('\t')
            #text = text.split()
            # 空白要素は捨てる
            #text = [t.strip() for t in text if t]
            # 一単元の文書が偶数個の文章から成るようにする(BERTのデータセットの都合上)
            max_text_len = len(text) // 2
            text = text[:max_text_len * 2]
            text = '\n'.join(text)
            f.write(text)
            count_byte += get_byte_num(text)
            if count_byte >= LIMIT_BYTE:
                break

0 bytes
83 bytes
143 bytes
241 bytes
309 bytes
404 bytes
494 bytes
652 bytes
756 bytes
881 bytes
983 bytes
1063 bytes
1156 bytes
1296 bytes
1409 bytes
1513 bytes
1591 bytes
1732 bytes
1827 bytes
1883 bytes
1993 bytes
2096 bytes
2228 bytes
2298 bytes
2397 bytes
2514 bytes
2646 bytes
2770 bytes
2869 bytes
2937 bytes
3020 bytes
3117 bytes
3233 bytes
3345 bytes
3424 bytes
3567 bytes
3660 bytes
3813 bytes
3875 bytes
4021 bytes
4140 bytes
4228 bytes
4325 bytes
4452 bytes
4542 bytes
4597 bytes
4705 bytes
4851 bytes
4929 bytes
5015 bytes
5105 bytes
5188 bytes
5259 bytes
5326 bytes
5464 bytes
5519 bytes
5639 bytes
5708 bytes
5816 bytes
5901 bytes
6019 bytes
6143 bytes
6270 bytes
6360 bytes
6449 bytes
6560 bytes
6686 bytes
6780 bytes
6863 bytes
6964 bytes
7057 bytes
7184 bytes
7316 bytes
7432 bytes
7506 bytes
7617 bytes
7715 bytes
7800 bytes
7898 bytes
8011 bytes
8113 bytes
8235 bytes
8322 bytes
8418 bytes
8523 bytes
8600 bytes
8743 bytes
8879 bytes
8945 bytes
9072 bytes
9155 bytes
9252 bytes
93

201697 bytes
201854 bytes
201938 bytes
202034 bytes
202164 bytes
202294 bytes
202382 bytes
202535 bytes
202644 bytes
202771 bytes
202871 bytes
202989 bytes
203088 bytes
203207 bytes
203313 bytes
203474 bytes
203580 bytes
203694 bytes
203797 bytes
203921 bytes
204027 bytes
204150 bytes
204349 bytes
204428 bytes
204543 bytes
204637 bytes
204743 bytes
204817 bytes
204908 bytes
205032 bytes
205111 bytes
205256 bytes
205376 bytes
205549 bytes
205625 bytes
205713 bytes
205798 bytes
205888 bytes
205999 bytes
206109 bytes
206217 bytes
206335 bytes
206434 bytes
206578 bytes
206698 bytes
206814 bytes
206908 bytes
207008 bytes
207114 bytes
207230 bytes
207329 bytes
207463 bytes
207551 bytes
207662 bytes
207771 bytes
207826 bytes
207934 bytes
208043 bytes
208149 bytes
208237 bytes
208342 bytes
208420 bytes
208545 bytes
208658 bytes
208770 bytes
208900 bytes
209026 bytes
209137 bytes
209224 bytes
209337 bytes
209431 bytes
209556 bytes
209662 bytes
209751 bytes
209826 bytes
209932 bytes
210053 bytes

376762 bytes
376838 bytes
376958 bytes
377040 bytes
377124 bytes
377233 bytes
377336 bytes
377415 bytes
377538 bytes
377642 bytes
377752 bytes
377864 bytes
378007 bytes
378136 bytes
378246 bytes
378352 bytes
378465 bytes
378588 bytes
378740 bytes
378873 bytes
378962 bytes
379067 bytes
379141 bytes
379284 bytes
379423 bytes
379492 bytes
379600 bytes
379675 bytes
379805 bytes
379925 bytes
380101 bytes
380224 bytes
380339 bytes
380396 bytes
380520 bytes
380588 bytes
380682 bytes
380815 bytes
380927 bytes
381035 bytes
381140 bytes
381218 bytes
381318 bytes
381450 bytes
381579 bytes
381685 bytes
381859 bytes
381974 bytes
382109 bytes
382214 bytes
382362 bytes
382473 bytes
382597 bytes
382712 bytes
382811 bytes
382943 bytes
383047 bytes
383123 bytes
383211 bytes
383288 bytes
383426 bytes
383539 bytes
383607 bytes
383685 bytes
383782 bytes
383896 bytes
384033 bytes
384129 bytes
384230 bytes
384290 bytes
384426 bytes
384546 bytes
384660 bytes
384758 bytes
384911 bytes
385004 bytes
385154 bytes

これでBERTの学習に使うデータセットができました.<br>
今度はTraining用とValidation用のデータに分割します.

In [32]:
num_lines = sum(1 for line in open(save_file))
print('Base file lines : ', num_lines)
# 全体の80%をTraining dataに当てます
train_lines = int(num_lines * 0.8)
print('Train file lines : ', train_lines)

Base file lines :  10000
Train file lines :  8000


dataは前処理済みテキスト保存場所 <br>
outputは訓練モデル保存場所として作成

In [9]:
! mkdir -p data output

In [33]:
out_file_name_temp = './data/chatdata/splitted_%d.txt'

split_index = 1
line_index = 1
out_file = open(out_file_name_temp % (split_index,), 'w')
in_file = open(save_file)
line = in_file.readline()
while line:
    if line_index > train_lines:
        print('Starting file: %d' % split_index)
        out_file.close()
        split_index = split_index + 1
        line_index = 1
        out_file = open(out_file_name_temp % (split_index,), 'w')
    out_file.write(line)
    line_index = line_index + 1
    line = in_file.readline()
    
out_file.close()
in_file.close()

Starting file: 1


In [34]:
print('Train file lines : ', sum(1 for line in open('./data/chatdata/splitted_1.txt')))
print('Valid file lines : ', sum(1 for line in open('./data/chatdata/splitted_2.txt')))

Train file lines :  8000
Valid file lines :  2000


これにてテキストの前処理は完了です！